In [1]:
import numpy as np
import math

In [2]:
def calculate_equivalence_classes(data, condition_attrs):
    data_indexed = data.set_index(condition_attrs)
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

In [3]:
from math import log

def getFitness(data,condition_attrs,decision_attr):
    # 计算D的信息熵
    total_records = len(data)

    equivalence_classes = calculate_equivalence_classes(data,condition_attrs)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)
    total_class=0
    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        if(class_size>1):
            for key_decision in equivalence_classes_dec.keys():
                x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
                if(len(x)==class_size):
                    total_class+=class_size
    avg_confidence = 1- total_class/total_records +0.1*log(len(feature_columns))

    return avg_confidence

In [4]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import pandas as pd

# entropy_DC=calculate_contidion_entropy(data,feature_columns,'target')
data = pd.read_csv('data/heart.csv')
feature_columns = [ col for col in data.columns if col not in ['id','target']]

target='target'
loop=1
initialize=getFitness(data,feature_columns,'target')
print(initialize)
while loop:
    mintemp=999
    for col in feature_columns:
        s=feature_columns.copy()
        s.remove(col)
        t=getFitness(data,s,'target')
        if(t<mintemp):
            mintemp=t
            a0=col
    if mintemp<=initialize:
        initialize=mintemp
        feature_columns.remove(a0)
    else:
        loop=0
print(feature_columns)

1.249894275680147
['fbs', 'restecg', 'ca', 'thal']


In [5]:
B=feature_columns.copy()

In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

df = pd.read_csv('data/heart.csv')
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id', 'target']]
Y = df["target"]
X = df[B]
def SVM(trainX, trainY, testX, testY):
    scaler = StandardScaler()
    scaler.fit(trainX)
    trainX3 = scaler.transform(trainX)
    testX3 = scaler.transform(testX)
    clf = svm.SVC(C=10, gamma=0.8, max_iter=800)
    clf.fit(trainX3, trainY)
    test_result = clf.predict(testX3)
    precision = np.sum(test_result == testY) / testY.shape[0]
    return precision

        # Use stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = []
for train_index, test_index in skf.split(X, Y):
    trainX, testX = X.iloc[train_index], X.iloc[test_index]
    trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
    score = SVM(trainX, trainY, testX, testY)
    scores.append(score)
avg_score = np.mean(scores)
print(avg_score)

0.8017204301075269


In [7]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

df = pd.read_csv('data/heart.csv')
feature_columns = [ col for col in df.columns if col not in ['id']]
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id','target']]
X = df[B]
Y = df['target']
dt = DecisionTreeClassifier(max_depth=5)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

accuracy_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

    dt.fit(X_train, y_train)

    y_pred = dt.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(mean_accuracy)

0.7855913978494623


In [8]:
from sklearn.neighbors import KNeighborsClassifier

k = 5
knn = KNeighborsClassifier(n_neighbors=k)
df = pd.read_csv('data/heart.csv')
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id','target']]
Y=df["target"]
X=df[B]
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracy_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(mean_accuracy)


0.7450537634408602
